# Formulations

Introduce Binary Integer varibales $C_i$ for each node. IF there is no edge between two nodes i and j, the constraints can be added as $C_i + C_j <=1$ telling that nodes i and j cannot be part of the same clique. 

$$
\begin{aligned}
& \underset{x}{\text{maximize}}
& & \sum_{i=1}^{|V|} C_i \\
& \text{subject to}
& & C_i + C_j \leq 1 , \forall (i,j) \not\in E \\ 
\end{aligned}
$$

In [1]:
import numpy as np
import gurobipy as gp
from gurobipy import GRB
import time
import pandas as pd
import json

In [2]:
def findMaximalClique(Adj,model, xp=None,sharing=None):
    """
    Finds Maximal Clique for given Adjacency Matrixl 
    
    Params:
    Adj: Adjacency matrix
    
    model: GurobiPi Model instance
    
    xp : binary numpy array, denoting previous clique.
    
    sharing: `'none'` to not include any nodes that are included from xp. `'allow'` to allow sharing of nodes from previous c
    clique xp
    
    Returns:
    C: tupleDict containing indices and corresponding values. if C[i] is 1, then 
    node `i` is included in clique K.
    """
    
    n_nodes = Adj.shape[0]
    
    idxs = np.argwhere((np.tril(Adj,-1)+np.triu(np.ones((n_nodes,1))*np.ones((1,n_nodes))))==0)

    
    C = model.addMVar(n_nodes, vtype=GRB.BINARY, name="C")
    constraints = model.addConstrs((C[i] + C[j]<=1) for i,j in idxs)
    if xp is not None:
        if sharing=='none':
            print('here')
            xp_idxs = np.argwhere(np.array(xp)==1)
            constraints2 = model.addConstr(C[xp_idxs].sum() == 0 )
            
            
        elif sharing == 'allow':
            xp_idxs = np.argwhere(np.array(xp)==0)
            constraints2 = model.addConstr(C[xp_idxs].sum() >= 1 )
            
        else:
            raise ValueError("Please Specify a valid value for `sharing` parameter. See function signature for more information")
            
    
    
    model.setObjective(gp.quicksum(C) , GRB.MAXIMIZE )
    model.optimize()
    
    return C
    
    

In [3]:
def findAllMaximalCliques(Adj,sharing='none'):
    """
    Finds all maximal cliques given From a graph given by Adj.
    
    Params:
    
    Adj: Adjacency matrix of the graph
    
    sharing: `'none'` to not include any nodes that are included from xp. `'allow'` to allow sharing of nodes from previous c
    clique xp
    
    Returns:
    
    Cliques: MxN array, where M is hte number of cliques found, and N is the number of Nodes. 
    
    """
    N = Adj.shape[0]
    a = np.array([1])
    xp = np.zeros(N)
    cliques = []

    while True:
        print("Finding Another Maximal Clique....")
        #Build Gurobi Model
        model = gp.Model('maximalClique')
        a = findMaximalClique(Adj,model,xp,sharing='allow')

        xp = xp+a.x
        print(".....................................")

        if a.x.sum()<=2:
            break
        cliques.append(a.x)
        print(f"Clique of size {a.x.sum()} found")
        print(".....................................")
        
    return np.array(cliques),model

In [4]:
import re
lp_file = []
with open("nc.lp") as f:
    for line in f:
        line=re.sub('Maximize','Minimize',line)
        
        line=re.sub("[\[\]']", '', line) 
        lp_file.append(line)
lp_file[3] = re.sub('[\+]' , '-' , lp_file[3])
lp_file[3] = re.sub('[^C0]' , '-' , lp_file[3],count=1)

with open('nc.lp', 'w') as f:
    for line in lp_file:
        f.write(line)

## Near Clique Problem

We can relax teh above constriants by adding a binary variable $V_{ij}$ for every constraint between i and j 

$$C_i + C_j  - V_{ij}\leq 1 \forall (i,j) \not\in E$$

If a maximum of N edges can be added, then aditional constraint , $\sum V_{ij} = N$ can be added. 

In [5]:
def findNearClique(Adj,model,N,xp=None,sharing=None):
    """
    Finds Near Clique for given adjacency matrix.
    
    Params:
    --------------
    Adj: numpy arrya of adjaceny matrix
    
    model: Gurobipy model
    
    N: Maximum number of edges that should be added
    
     xp : binary numpy array, denoting previous clique.
    
    sharing: `'none'` to not include any nodes that are included from xp. 
    `'allow'` to allow sharing of nodes from previous clique xp
    
    Returns:
    C: Binary Vairable denoting the clique K
    
    V: Binary Variables coresponding to formation of edges
    
    idx: Kx2 where K is the number of constraints. Gives edges corresponding to each of the binary variable in V
    -------------
    """
    
    n_nodes = Adj.shape[0]
    
    idxs = np.argwhere((np.tril(Adj,-1)+np.triu(np.ones((n_nodes,1))*np.ones((1,n_nodes))))==0)
    n_constraints = idxs.shape[0]
  
    

    
    C = model.addMVar(n_nodes, vtype=GRB.BINARY, name="C")
    V = model.addMVar(n_constraints, vtype=GRB.BINARY, name="V")

    constraints1 = model.addConstrs((C[i] + C[j] - V[idx] <=1) for idx,(i,j) in enumerate(idxs))
    constraints2 = model.addConstr(V.sum()==N)
    
    if xp is not None:
        if sharing=='none':
            
            xp_idxs = np.argwhere(np.array(xp)==1)
            constraints3 = model.addConstr(C[xp_idxs].sum() == 0 )
            
            
        elif sharing == 'allow':
            xp_idxs = np.argwhere(np.array(xp)==0)
            constraints3 = model.addConstr(C[xp_idxs].sum() >= 1 )
            
        else:
            raise ValueError("Please Specify a valid value for `sharing` parameter. See function signature for more information")
            
    
    model.setObjective(gp.quicksum(C) , GRB.MAXIMIZE )
    model.optimize()
    return C,V,idxs

In [6]:
def findAllNearCliques(Adj,n,sharing='none'):
    
    """
    Completes all near cliques from a graph given by Adj. 
    
    Params:
    Adj: Adjacency Matrix of the graph
    
    n: Number of edges that can be added
    
    sharing: `'none'` denotes none of the nodes from the previous cliques can be shared, `'allow'` denotes allow  sharing 
    between nodes of the previous cliques.
    
    Returns:
    
    Cliques: NDArray of Cliques. It is MxN matrix, where M is the number of lciques found and N is the number of nodes in
    the graph.
    
    Vs: ND array of MxE,  where E is the cardinality of the set `Not(E)`. M is the number of cliques found.
    
    idx: NDArray of size Ex2, where E is the cardinaluity of the set `not(E)`. 
    
    idx[Vs[i,:]==1] Gives the edges that are added in ith Clique. 

    """
    N = Adj.shape[0]
    a = np.array([1])
    xp = np.zeros(N)
    cliques = []
    Vs = []
    idxs = []

    while True:
        print("Completing Another Near Clique....")
        #Build Gurobi Model
        model = gp.Model('nearClique')
        a,V,idx = findNearClique(Adj,model,n,xp,sharing=sharing)

        xp = xp+a.x
        print(".....................................")
        

        if a.x.sum()<=2 or sum(xp)>N:
            break
        cliques.append(a.x)
        Vs.append(V.x)
        #idxs.append(idx)
        print(f"Clique of size {a.x.sum()} found")
        print(".....................................")
        
    return np.array(cliques),np.array(Vs),np.array(idx)

## Inverse Near Clique Problem
Given a graph G(E,V) and it has a maximal clique of size k, Inverse clique problem is to find the minimum number of edges to be added to G, so that the maximal clique is of size atleast k+d , for some positive integer d

$$
\begin{aligned}
& \underset{x}{\text{Minimize}}
& & W  \\
& \text{subject to}
& & C_i + C_j  - V_{ij} \leq 1 , \forall (i,j) \not\in E \\ 
& & & \sum V_{ij} \leq W , \forall (i,j) \notin E \\ 
& & & \sum_i^{|V|} C_i \geq k+d
\end{aligned}
$$

In [7]:
def invNearClique(Adj,model,k,d,xp=None,sharing=None):
    """
    Given a graph G(E,V) and it has a maximal clique of size
    k, Inverse clique problem is to find the minimum number of
    edges to be added to G, so that the maximal clique is of size
    atleast k+d , for some positive integer d
    
    Params:
    --------------
    Adj: numpy arrya of adjaceny matrix
    
    model: Gurobipy model
    
    N: Maximum number of edges that should be added
    
     xp : binary numpy array, denoting previous clique.
    
    sharing: `'none'` to not include any nodes that are included from xp. `'allow'` to allow sharing of nodes from previous c
    clique xp
    
    Returns:
    C: Binary Vairable denoting the clique K
    
    V: Binary Variables coresponding to formation of edges
    
    idx: Kx2 where K is the number of constraints. Gives edges corresponding to each of the binary variable in V
    -------------
    """
    
    n_nodes = Adj.shape[0]
    
    idxs = np.argwhere((np.tril(Adj,-1)+np.triu(np.ones((n_nodes,1))*np.ones((1,n_nodes))))==0)
    n_constraints = idxs.shape[0]
  
    

    
    C = model.addMVar(n_nodes, vtype=GRB.BINARY, name="C")
    V = model.addMVar(n_constraints, vtype=GRB.BINARY, name="V")
    W = model.addMVar(1 , vtype=GRB.INTEGER,name="W")
    print("setting up constraints...")
    constraints1 = model.addConstrs((C[i] + C[j] - V[idx] <=1) for idx,(i,j) in enumerate(idxs))
    constraints2 = model.addConstr(C.sum()>=k+d)
   # print(W.shape)
    constraints3 = model.addConstr(V.sum()<=W.sum() )
    
    if xp is not None:
        if sharing=='none':
            
            xp_idxs = np.argwhere(np.array(xp)==1)
            constraints4 = model.addConstr(C[xp_idxs].sum() == 0 )
            
            
        elif sharing == 'allow':
            xp_idxs = np.argwhere(np.array(xp)==0)
            constraints4 = model.addConstr(C[xp_idxs].sum() >= 1 )
            
        else:
            raise ValueError("Please Specify a valid value for `sharing` parameter. See function signature for more information")
            
    
    model.setObjective(W.sum() , GRB.MINIMIZE )
    print("optimizing....")
    model.optimize()
    return C,V,idxs

In [8]:
def findAllInvNearCliques(Adj,k,d,sharing='none'):
    
    """
    Completes all near cliques from a graph given by Adj. 
    
    Params:
    Adj: Adjacency Matrix of the graph
    
    k:
    
    d:
    
    sharing: `'none'` denotes none of the nodes from the previous cliques can be shared, `'allow'` denotes allow  sharing 
    between nodes of the previous cliques.
    
    Returns:
    
    Cliques: NDArray of Cliques. It is MxN matrix, where M is the number of lciques found and N is the number of nodes in
    the graph.
    
    Vs: ND array of MxE,  where E is the cardinality of the set `Not(E)`. M is the number of cliques found.
    
    idx: NDArray of size Ex2, where E is the cardinaluity of the set `not(E)`. 
    
    idx[Vs[i,:]==1] Gives the edges that are added in ith Clique. 

    """
    N = Adj.shape[0]
    a = np.array([1])
    xp = np.zeros(N)
    cliques = []
    Vs = []
    idxs = []

    while True:
        print("Completing Another Near Clique....")
        #Build Gurobi Model
        model = gp.Model('invnearClique')
        a,V,idx = invNearClique(Adj,model,k,d,xp,sharing=sharing)

        xp = xp+a.x
        print(".....................................")
        

        if a.x.sum()<=2 or sum(xp)>N:
            break
        cliques.append(a.x)
        Vs.append(V.x)
        #idxs.append(idx)
        print(f"Clique of size {a.x.sum()} found")
        print(".....................................")
        
    return np.array(cliques),np.array(Vs),np.array(idx)

In [28]:
import numpy as np
import pandas as pd
import time
df = pd.read_excel("../CliqueExps/Matlab/exp56.xlsx",header=None)
Adj = df.to_numpy()

model = gp.Model('invClique')

start = time.time()
C,V,idx = invNearClique(Adj, model,16,15)
end = time.time()

print("time: ", end-start)
print(C.x.sum())






Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-09
setting up constraints...
optimizing....
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 1408 rows, 1463 columns and 5681 nonzeros
Model fingerprint: 0xd420f16b
Variable types: 0 continuous, 1463 integer (1462 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 3e+01]
Presolve removed 1 rows and 1 columns
Presolve time: 0.02s
Presolved: 1407 rows, 1462 columns, 4274 nonzeros
Variable types: 0 continuous, 1462 integer (1462 binary)
Found heuristic solution: objective 331.0000000

Root relaxation: objective 1.170000e+02, 3746 iterations, 0.20 seconds (0.15 work units)

    Nodes    |    Current N

# Usage

To Show the Usage of above functions, A toy problem is depicted here.
Consider the toy problem here

![image.png](toy.png)

The Adjacency matrix for the graph can be given by 
$$
Adj = 
\begin{bmatrix}
1 & 1 & 0 & 0 & 0 & 1 \\
1 & 1 & 1 & 1 & 0 & 0 \\
0 & 1 & 1 & 1 & 0 & 0 \\
0 & 1 & 1 & 1 & 1 & 0 \\
0 & 0 & 0 & 1 & 1 & 1\\
1 & 0 & 0 & 0 & 1 & 1 \\
\end{bmatrix}
$$

In [9]:
Adj = np.array([[1,1,0,0,0,1],
      [1,1,1,1,0,0],
      [0,1,1,1,0,0],
       [0,1,1,1,1,0],
       [0,0,0,1,1,1],
       [1,0,0,0,1,1]])
N = Adj.shape[0]

In [10]:
#Find one maximal clique
model = gp.Model('maximalClique')
a = findMaximalClique(Adj,model)


Set parameter Username
Academic license - for non-commercial use only - expires 2022-04-24
Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (linux64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 8 rows, 6 columns and 16 nonzeros
Model fingerprint: 0x3d0ec776
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 13 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)

Root relaxation: objective 3.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       3.000

In [11]:
model.write("mc.mps")

In [31]:
a.x

array([ 0.,  1.,  1.,  1., -0., -0.])

In [12]:
clqs=findAllMaximalCliques(Adj,'allow')

Finding Another Maximal Clique....
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 9 rows, 6 columns and 22 nonzeros
Model fingerprint: 0x88df882b
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 3 rows and 0 columns
Presolve time: 0.00s
Presolved: 6 rows, 6 columns, 13 nonzeros
Variable types: 0 continuous, 6 integer (6 binary)

Root relaxation: objective 3.000000e+00, 1 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       3.0000000    3.00000  0.00%     -    0s

Explored 1 nodes (1 si

In [13]:
clqs

array([[ 0.,  1.,  1.,  1., -0., -0.]])

### finding Near Clique

In [34]:
model = gp.Model("nealClique")
C,V,idx = findNearClique(Adj,model,2)
model.write("nc.lp")

Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 9 rows, 14 columns and 32 nonzeros
Model fingerprint: 0x8f6318e4
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 3.0000000
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 32 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)

Root relaxation: objective 4.000000e+00, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.0000000    4.00000  0.00%     -    0s

Explored 1 nodes (12 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count 

In [64]:
import re
lp_file = []
with open("nc.lp") as f:
    for line in f:
        line=re.sub('Maximize','Minimize',line)
        
        line=re.sub("[\[\]']", '', line) 
        lp_file.append(line)
lp_file[3] = re.sub('[\+]' , '-' , lp_file[3])
lp_file[3] = re.sub('[^C0]' , '-' , lp_file[3],count=1)

with open('nc.lp', 'w') as f:
    for line in lp_file:
        f.write(line)

# Gold Standard



In [14]:
Adj = np.loadtxt("../../Matlab/data/GS_56x56.txt")

In [17]:
model = gp.Model("maxClique")
C = findMaximalClique(Adj,model)


Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 927 rows, 56 columns and 1854 nonzeros
Model fingerprint: 0x236d19be
Variable types: 0 continuous, 56 integer (56 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 31.0000000
Presolve removed 927 rows and 56 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.01 work units)
Thread count was 1 (of 16 available processors)

Solution count 1: 31 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.100000000000e+01, best bound 3.100000000000e+01, gap 0.0000%


In [23]:
model.write("GS_56.lp")

In [24]:
import re
lp_file = []
with open("GS_56.lp") as f:
    for line in f:
        line=re.sub('Maximize','Minimize',line)
        line = re.sub('R\d+:','',line)
       # line = re.sub(':','',line)
        
        line=re.sub("[\[\]']", '', line) 
        lp_file.append(line)
lp_file[3] = re.sub('[\+]' , '-' , lp_file[3])
lp_file[3] = re.sub('[^C0]' , '-' , lp_file[3],count=1)

with open('GS_56x56.lp', 'w') as f:
    for line in lp_file:
        f.write(line)

In [32]:
C,V,idx = findAllNearCliques(Adj,2,sharing='allow')

Completing Another Near Clique....
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 10 rows, 14 columns and 38 nonzeros
Model fingerprint: 0x72254f84
Variable types: 0 continuous, 14 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 2e+00]
Found heuristic solution: objective 3.0000000
Presolve removed 1 rows and 0 columns
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 32 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)

Root relaxation: objective 4.000000e+00, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

*    0     0               0       4.0000000    4.00000  0.00%     -    0s

Explored 1 nod

In [15]:
C

array([[-0.,  1.,  1.,  1.,  1.,  0.]])

In [17]:
C,V,idx = findAllInvNearCliques(Adj,3,1,sharing="allow")

Completing Another Near Clique....
setting up constraints...
optimizing....
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 11 rows, 15 columns and 45 nonzeros
Model fingerprint: 0xc0fd6027
Variable types: 0 continuous, 15 integer (14 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 4e+00]
Found heuristic solution: objective 2.0000000
Presolve removed 2 rows and 1 columns
Presolve time: 0.00s
Presolved: 9 rows, 14 columns, 30 nonzeros
Variable types: 0 continuous, 14 integer (14 binary)

Root relaxation: cutoff, 12 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0     cutoff    0         2.00000    2.00000  0.00%   

In [103]:
C

array([[-0.,  0., -0., ..., -0.,  0.,  0.],
       [-0., -0., -0., ...,  0.,  0.,  0.],
       [-0.,  0., -0., ..., -0.,  0.,  0.],
       ...,
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0., -0.],
       [ 0.,  0.,  0., ...,  0.,  0., -0.]])

# Yeast : 209 Node

In [10]:
import numpy as np
import pandas as pd 
with open("../../Gusfield-Archive/yeastAdjMatrix") as f:
    mat = []
    for line in f:
        lne = []
        for char in line:
            if char!='\n':
                lne.append(int(char))
        mat.append(lne)

In [11]:
mat = np.array(mat)

In [12]:
mat.shape

(209, 209)

In [13]:
model = gp.Model("Yeast_209")
findMaximalClique(mat,model)

Set parameter Username

--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only - expires 2022-03-09
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 19960 rows, 209 columns and 39920 nonzeros
Model fingerprint: 0x1287fce4
Variable types: 0 continuous, 209 integer (209 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 7.0000000
Presolve removed 19486 rows and 13 columns
Presolve time: 0.14s
Presolved: 474 rows, 196 columns, 11562 nonzeros
Variable types: 0 continuous, 196 integer (196 binary)

Root relaxation: objective 1.310860e+01, 349 iterations, 0.02 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Wo

<(209,) matrix variable>

In [22]:
model.write("Yeast.lp")

In [25]:
import re
lp_file = []
with open("Yeast.lp") as f:
    for line in f:
        line=re.sub('Maximize','Minimize',line)
        line = re.sub('R\d+:','',line)
       # line = re.sub(':','',line)
        
        line=re.sub("[\[\]']", '', line) 
        lp_file.append(line)
lp_file[3] = re.sub('[\+]' , '-' , lp_file[3])
lp_file[3] = re.sub('[^C0]' , '-' , lp_file[3],count=1)

with open('yeast_2099.lp', 'w') as f:
    for line in lp_file:
        f.write(line)

In [95]:
C = findAllMaximalCliques(mat,sharing='allow')

Finding Another Maximal Clique....
Gurobi Optimizer version 9.5.0 build v9.5.0rc5 (win64)
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads
Optimize a model with 19961 rows, 209 columns and 40129 nonzeros
Model fingerprint: 0x2f906004
Variable types: 0 continuous, 209 integer (209 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 6.0000000
Presolve removed 19487 rows and 8 columns
Presolve time: 0.15s
Presolved: 474 rows, 201 columns, 12672 nonzeros
Variable types: 0 continuous, 201 integer (201 binary)

Root relaxation: objective 1.311831e+01, 348 iterations, 0.01 seconds (0.02 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0   13.11830    0   85    6.00000   13.11830   119%     -

In [105]:
C.shape

(132, 209)

### Random Graphs: Maximal Clique Finding

In [106]:
df1 = pd.read_excel("./100_05.xlsx",header=None)
df2 = pd.read_excel("./300_25.xlsx",header=None)
df3 = pd.read_excel("./500_25.xlsx",header=None)

exp_name  = ["Random_100_05","Random_300_25","Random_500_25"]
T = []


model = gp.Model(exp_name[0])
start = time.time()
findMaximalClique(df1.to_numpy(),model=model)
end = time.time()
T.append(end-start)

model = gp.Model(exp_name[1])
start = time.time()
findMaximalClique(df2.to_numpy(),model=model)
end = time.time()
T.append(end-start)

model = gp.Model(exp_name[2])
start = time.time()
findMaximalClique(df3.to_numpy(),model=model)
end = time.time()
T.append(end-start)

for exp_n,t in zip(exp_name,T):
	res = {"time": t}
	meta[exp_n] = res



# Complete Yeast Dataset

In [126]:

nodes = []
with open("./Spoke_TAP_format.txt") as f:
    for line in f:
        edge = line.strip().split("\t")
        if edge[0] not in nodes:
            nodes.append(edge[0])
        if edge[1] not in nodes:
            nodes.append(edge[1])
        

In [128]:
N = len(nodes)
Adj = np.zeros((N,N))

with open("./Spoke_TAP_format.txt") as f:
    for line in f:
        edge = line.strip().split("\t")
        
        n1_idx = nodes.index(edge[0])
        n2_idx = nodes.index(edge[1])
        
        Adj[n1_idx,n2_idx] = 1
        Adj[n2_idx,n1_idx] = 1

In [129]:
print(f"Graph with {Adj.shape[0]} Nodes and {int(Adj.sum()/2)} Edges")

Graph with 2283 Nodes and 6645 Edges


In [138]:
nodes = np.array(nodes)
np.savetxt("MIPS_nodeList.csv", nodes, delimiter=",",fmt='%s')

## Find All Maximal Cliques 

In [ ]:
start = time.time()
C = findAllMaximalCliques(Adj,sharing='allow')
end = time.time()
t = end-start
print(f"Total of {C.shape[0]} Maximal Cliques Found in {t}s.")

C = np.array(C)

numpy.savetxt("MIPS_Maximal_Cliques.csv", C, delimiter=",")

with open("MIPS_Maximal_Cliques_meta.txt") as f:
    f.write(f"Number of Cliques,{C.shape[0]}\n")
    f.write(f"time,{t}\n")


In [123]:
Adj

array([[0., 1., 0., ..., 0., 0., 0.],
       [1., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

### Predicting Edges

For Each Size Clique in C, we shall try to complete the clique by inclusion of atmost 7 nodes. 

In [150]:
Ks = np.unique(C.sum(axis=1))
extra_nodes = np.arange(1,8)

In [ ]:

results = {}

for k in Ks:
    for d in extra_nodes:
        start = time.time()
        clqs,Vs,idx = findAllInvNearCliques(Adj,k,d,sharing='allow')
        end = time.time()
        
        res =  {"Cliques" : clqs ,
               "Vs": Vs,
               "idx":idx,
               "time":end-start}
        results[k,d] = res

with open('MIPS_Pred.txt', 'w') as f:
     f.write(json.dumps(results))          